In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm
import pandas as pd
from sklearn.metrics import mean_absolute_error
from copy import deepcopy
sns.set()

In [2]:
X_train = pd.read_csv('../input/project/lt_train.csv/lt_train.csv')
X_test = pd.read_csv('../input/project/lt_test.csv')

In [3]:
df = X_train.loc[X_train['demand'].isnull() == False]

In [4]:
#посмотрим на период данных для обучения
print(df['period_dt'].min())
print(df['period_dt'].max())

2016-12-19
2018-12-24


In [5]:
#посмотрим на период данных для предсказания
print(X_test['period_dt'].min())
print(X_test['period_dt'].max())

2019-01-07
2019-12-30


In [6]:
X_train['Date'] = pd.to_datetime(X_train['period_dt'], dayfirst = True)
X_train.set_index(['Date'], inplace=True)
X_train.drop('period_dt', axis=1, inplace=True)
X_train.head()

,location_id,product_id,demand,PROMO1_FLAG,PROMO2_FLAG,PRICE_REGULAR,PRICE_AFTER_DISC,NUM_CONSULTANT,AUTORIZATION_FLAG,id
Date,,,,,,,,,,
2019-12-16,764.0,23285.0,NaN,0.0,0.0,2199.0,1099.5,0.0,1.0,1
2019-12-30,764.0,23285.0,NaN,0.0,0.0,2199.0,1099.5,0.0,1.0,3
2019-12-09,764.0,23285.0,NaN,0.0,0.0,2199.0,1099.5,0.0,0.0,4
2019-12-02,453.0,23285.0,NaN,1.0,0.0,2199.0,1319.4,0.0,1.0,5
2019-12-02,764.0,23285.0,NaN,1.0,0.0,2199.0,1099.5,0.0,0.0,6


In [7]:
#создадим признаки, обозначающие день, месяц и год для наблюдения
data = X_train.reset_index().copy(deep = True)

data["monthday"] = data.Date.dt.day
data["month"] = data.Date.dt.month
data["year"] = data.Date.dt.year

#прологарифмируем целевую переменную (без логарифма работает хуже) 
data = data.assign(ln_demand = np.log(data['demand'] + 1)) 

In [8]:
data.drop('Date', axis=1, inplace=True)

In [9]:
#заполним пропуски в спросе 
data['demand'] = data['demand'].fillna(-10)

In [10]:
#создадим признак Promo, равный 1, если хотя бы один из вариантов промо присутствовал
#data['PROMO1_FLAG'] = data['PROMO1_FLAG'].fillna(0)
#data['PROMO2_FLAG'] = data['PROMO2_FLAG'].fillna(0)
#data['Promo'] = data['PROMO1_FLAG'] + data['PROMO2_FLAG']

#def scaler(x):
#    if x >=1:
#        x = 1
#    else:
#        x = 0 
#    return x 
#в PROMO1_FLAG есть значения больше 1, поэтому приравняем полученные суммы больше 1 к 1 
#data['Promo'] = data['Promo'].apply(lambda x: scaler(x))

In [11]:
#data = data.drop(['PROMO1_FLAG', 'PROMO2_FLAG'], axis=1)

In [12]:
#создадим переменную, обозначающую фактическую цену товара 
#data['actual_price'] = data['PRICE_REGULAR']
#data['actual_price'].loc[data['Promo'] == 1] = data['PRICE_AFTER_DISC']
#data = data.drop(['PRICE_REGULAR', 'PRICE_AFTER_DISC'], axis=1)
#data.head()

In [13]:
#data['AUTORIZATION_FLAG'] = data['AUTORIZATION_FLAG'].fillna(1)
#data['NUM_CONSULTANT'] = data['NUM_CONSULTANT'].fillna(0)

In [14]:
#data.isnull().sum()

In [15]:
#уберем наблюдения, по которым нет данных для товаров и магазинов 
#data = data.loc[data['product_id'].isnull() == False]
#data.isnull().sum()

In [16]:
#заполним оставшиеся пропуски нулями
#data = data.fillna(0)

In [17]:
data = data.fillna(0)
le = LabelEncoder()
le.fit(data['location_id'])
data['location_id'] = le.transform(data['location_id'])
le.fit(data['product_id'])
data['product_id'] = le.transform(data['product_id'])

In [18]:

sc = MinMaxScaler()
sc.fit(data.iloc[:, 3:9])
data.iloc[:, 3:9] = sc.transform(data.iloc[:, 3:9])

#этот вариант используется при создании признаков promo и actual price 
#sc.fit(data.iloc[:, 3:5])
#data.iloc[:, 3:5] = sc.transform(data.iloc[:, 3:5])

In [19]:
data

,location_id,product_id,demand,PROMO1_FLAG,PROMO2_FLAG,PRICE_REGULAR,PRICE_AFTER_DISC,NUM_CONSULTANT,AUTORIZATION_FLAG,id,monthday,month,year,ln_demand
0,19,6,-10.0,0.000000,0.0,0.385857,0.201780,0.0,1.0,1,16,12,2019,0.0
1,19,6,-10.0,0.000000,0.0,0.385857,0.201780,0.0,1.0,3,30,12,2019,0.0
2,19,6,-10.0,0.000000,0.0,0.385857,0.201780,0.0,0.0,4,9,12,2019,0.0
3,6,6,-10.0,0.333333,0.0,0.385857,0.242136,0.0,1.0,5,2,12,2019,0.0
4,19,6,-10.0,0.333333,0.0,0.385857,0.201780,0.0,0.0,6,2,12,2019,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3404841,0,0,-10.0,0.333333,0.0,0.000000,0.000000,0.0,0.0,3409994,30,12,2019,0.0
3404842,0,0,-10.0,0.333333,0.0,0.000000,0.000000,0.0,0.0,3409995,30,12,2019,0.0
3404843,0,0,-10.0,0.333333,0.0,0.000000,0.000000,0.0,0.0,3409996,30,12,2019,0.0
3404844,0,0,-10.0,0.333333,0.0,0.000000,0.000000,0.0,0.0,3409997,30,12,2019,0.0


In [20]:
#добавление лаговых фичей ухудшает результат, поэтому не используется в итоговой версии 

#def lag_features(dataframe, lags):
#    for lag in lags:
#        dataframe['demand_lag_' + str(lag)] = dataframe.groupby(["location_id", "product_id"])['demand'].transform(
#            lambda x: x.shift(lag))
#    return dataframe

#df = lag_features(data, [35, 87, 52])

In [21]:
#для случая использования лаговых фичей 
#data_test = df
#data = df.loc[df['demand'] > -1]

data_test = data
data = data.loc[data['demand'] > -1]

In [22]:
#найдем новые товары 
data_new_id = []

for i in tqdm(range(1, 42)):
    data_0 = data[(data['location_id'] == i)]
    news = data_0['product_id'].tolist()
    for j in range(1, len(news)):
        if j in news:
            data_1 = data_0[(data_0['product_id'] == j)]
            if data_1.shape[0] < 10:
                data_new_id.extend(data_1['id'].tolist())
            else:
                data_1 = data_1.loc[0:10, :]
                data_new_id.extend(data_1['id'].tolist())

100%|██████████| 41/41 [00:37<00:00,  1.09it/s]


In [23]:
#оставим в обучающей выборке только новые товары 
data = data[data['id'].isin(data_new_id)]
test_id_list = X_test['id'].tolist()

In [24]:
data['demand'].describe()
#возьмем порог 0.7 для высокого спроса (медиана)

count    50570.000000
mean         0.710086
std          0.929250
min          0.000000
25%          0.036458
50%          0.700000
75%          1.000000
max         27.000000
Name: demand, dtype: float64

In [25]:
#создадим признак, равный 1, если спрос был высокий, и 0 иначе
data['is_high'] = np.zeros(len(data))
data['is_high'].loc[data['demand'] >= 0.7] = 1
data_high = data.loc[data['is_high'] == 1]
data_low = data.loc[data['is_high'] == 0]
print(len(data_high))
print(len(data_low))

25292
25278


/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [26]:
data.drop('demand', axis=1, inplace=True)

In [27]:
#функция кросс-валидации
def CV(X_train, y_train, number_folds, model, metrics, kwargs={}):

    k = int(np.floor(float(X_train.shape[0]) / number_folds))

    errors = np.zeros(number_folds-1)
 
    for i in range(2, number_folds + 1):
        print('')
        split = float(i-1)/i

        X = X_train[:(k*i)]
        y = y_train[:(k*i)]

        index = int(np.floor(X.shape[0] * split))
       
        X_trainFolds = X[:index]        
        y_trainFolds = y[:index]

        X_testFold = X[(index + 1):]
        y_testFold = y[(index + 1):]

        model.fit(X_trainFolds, y_trainFolds, **kwargs, eval_set=[(X_testFold, y_testFold)])
        errors[i-2] = metrics(model.predict(X_testFold), y_testFold)
    
    return errors.mean()

In [28]:
#будем предсказывать переменную высокого спроса 
y_train_clf = data.is_high
X_train_clf = data.drop(['id','ln_demand','is_high'], axis=1) 

In [29]:
import lightgbm as lgb
import xgboost as xgb 
import catboost
from catboost import CatBoostClassifier

In [30]:
#для предсказания обучим классификатор 
#вариант 1 
lgbm_clf = lgb.LGBMClassifier(
                        n_estimators=350,
                        learning_rate=0.1,
                        feature_fraction=0.7,
                        subsample=0.4,
                        num_leaves=40)    
kwargs={'early_stopping_rounds':10,'verbose':2}

In [31]:
#вариант 2
catboost_clf = CatBoostClassifier(iterations=350,
                         learning_rate=0.1)

In [32]:
#вариант 3 
xgb_clf = xgb.XGBClassifier(n_estimators=350,
                        learning_rate=0.1)

In [33]:
CV(X_train_clf, y_train_clf, 20, lgbm_clf, roc_auc_score, kwargs)
CV(X_train_clf, y_train_clf, 20, catboost_clf, roc_auc_score, kwargs)
CV(X_train_clf, y_train_clf, 20, xgb_clf, roc_auc_score, kwargs)


[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[2]	valid_0's binary_logloss: 0.653394
[4]	valid_0's binary_logloss: 0.633113
[6]	valid_0's binary_logloss: 0.630684
[8]	valid_0's binary_logloss: 0.628493
[10]	valid_0's binary_logloss: 0.610217
[12]	valid_0's binary_logloss: 0.610096
[14]	valid_0's binary_logloss: 0.600038
[16]	valid_0's binary_logloss: 0.600035
[18]	valid_0's binary_logloss: 0.59687
[20]	valid_0's binary_logloss: 0.603209
[22]	valid_0's binary_logloss: 0.602936
[24]	valid_0's binary_logloss: 0.611711
[26]	valid_0's binary_logloss: 0.611481
[28]	valid_0's binary_logloss: 0.618929

[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[2]	valid_0's binary_logloss: 0.650349
[4]	valid_0's binary_logloss: 0.619583
[6]	valid_0's binary_logloss: 0.598706
[8]	valid_0's binary_logloss: 0.585028
[10]	valid_0's binary_logloss: 0.575498

/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecat

[8]	valid_0's binary_logloss: 0.555208
[10]	valid_0's binary_logloss: 0.529332
[12]	valid_0's binary_logloss: 0.515076
[14]	valid_0's binary_logloss: 0.500258
[16]	valid_0's binary_logloss: 0.490973
[18]	valid_0's binary_logloss: 0.485206
[20]	valid_0's binary_logloss: 0.480398
[22]	valid_0's binary_logloss: 0.474386
[24]	valid_0's binary_logloss: 0.465954
[26]	valid_0's binary_logloss: 0.461921
[28]	valid_0's binary_logloss: 0.458874
[30]	valid_0's binary_logloss: 0.454311
[32]	valid_0's binary_logloss: 0.453454
[34]	valid_0's binary_logloss: 0.44947
[36]	valid_0's binary_logloss: 0.445136
[38]	valid_0's binary_logloss: 0.442317
[40]	valid_0's binary_logloss: 0.442102
[42]	valid_0's binary_logloss: 0.439582
[44]	valid_0's binary_logloss: 0.438516
[46]	valid_0's binary_logloss: 0.438257
[48]	valid_0's binary_logloss: 0.440016
[50]	valid_0's binary_logloss: 0.439644
[52]	valid_0's binary_logloss: 0.437748
[54]	valid_0's binary_logloss: 0.437216
[56]	valid_0's binary_logloss: 0.43784
[58

/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecat

[34]	valid_0's binary_logloss: 0.471749
[36]	valid_0's binary_logloss: 0.469937
[38]	valid_0's binary_logloss: 0.467188
[40]	valid_0's binary_logloss: 0.461544
[42]	valid_0's binary_logloss: 0.459683
[44]	valid_0's binary_logloss: 0.45731
[46]	valid_0's binary_logloss: 0.454823
[48]	valid_0's binary_logloss: 0.453169
[50]	valid_0's binary_logloss: 0.453323
[52]	valid_0's binary_logloss: 0.450606
[54]	valid_0's binary_logloss: 0.449303
[56]	valid_0's binary_logloss: 0.449758
[58]	valid_0's binary_logloss: 0.448596
[60]	valid_0's binary_logloss: 0.452547
[62]	valid_0's binary_logloss: 0.452243
[64]	valid_0's binary_logloss: 0.450892
[66]	valid_0's binary_logloss: 0.451894
[68]	valid_0's binary_logloss: 0.454599

[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[2]	valid_0's binary_logloss: 0.641809
[4]	valid_0's binary_logloss: 0.593219
[6]	valid_0's binary_logloss: 0.565029
[8]	valid_0's binary_logloss: 0.543662


...

/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:797: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  UserWarning,


[6]	validation_0-logloss:0.59426
[8]	validation_0-logloss:0.58087
[10]	validation_0-logloss:0.57078
[12]	validation_0-logloss:0.56622
[14]	validation_0-logloss:0.56380
[16]	validation_0-logloss:0.56115
[18]	validation_0-logloss:0.55342
[20]	validation_0-logloss:0.55267
[22]	validation_0-logloss:0.55072
[24]	validation_0-logloss:0.54591
[26]	validation_0-logloss:0.54282
[28]	validation_0-logloss:0.53757
[30]	validation_0-logloss:0.53422
[32]	validation_0-logloss:0.53116
[34]	validation_0-logloss:0.53034
[36]	validation_0-logloss:0.52977
[38]	validation_0-logloss:0.52792
[40]	validation_0-logloss:0.52710
[42]	validation_0-logloss:0.52526
[44]	validation_0-logloss:0.52472
[46]	validation_0-logloss:0.52191
[48]	validation_0-logloss:0.52108
[50]	validation_0-logloss:0.51890
[52]	validation_0-logloss:0.51785
[54]	validation_0-logloss:0.51533
[56]	validation_0-logloss:0.51436
[58]	validation_0-logloss:0.51361
[60]	validation_0-logloss:0.51249
[62]	validation_0-logloss:0.51154
[64]	validation_

/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:797: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  UserWarning,


[6]	validation_0-logloss:0.53835
[8]	validation_0-logloss:0.51734
[10]	validation_0-logloss:0.50229
[12]	validation_0-logloss:0.49177
[14]	validation_0-logloss:0.48248
[16]	validation_0-logloss:0.47362
[18]	validation_0-logloss:0.46780
[20]	validation_0-logloss:0.46360
[22]	validation_0-logloss:0.45711
[24]	validation_0-logloss:0.45170
[26]	validation_0-logloss:0.44621
[28]	validation_0-logloss:0.44381
[30]	validation_0-logloss:0.44009
[32]	validation_0-logloss:0.43825
[34]	validation_0-logloss:0.43572
[36]	validation_0-logloss:0.43292
[38]	validation_0-logloss:0.43096
[40]	validation_0-logloss:0.42917
[42]	validation_0-logloss:0.42732
[44]	validation_0-logloss:0.42622
[46]	validation_0-logloss:0.42369
[48]	validation_0-logloss:0.42327
[50]	validation_0-logloss:0.42111
[52]	validation_0-logloss:0.42043
[54]	validation_0-logloss:0.41964
[56]	validation_0-logloss:0.41682
[58]	validation_0-logloss:0.41595
[60]	validation_0-logloss:0.41582
[62]	validation_0-logloss:0.41520
[64]	validation_

0.7796335348880689

In [34]:
data_test = data_test[data_test['id'].isin(test_id_list)]

In [35]:
demand_pred_lgbm = lgbm_clf.predict(data_test.drop(['id','demand','ln_demand'], axis=1))
demand_pred_catboost = catboost_clf.predict(data_test.drop(['id','demand','ln_demand'], axis=1))
demand_pred_xgb = xgb_clf.predict(data_test.drop(['id','demand','ln_demand'], axis=1))

In [36]:
#проверим первый вариант 
data_test['is_high'] = demand_pred_lgbm

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [37]:
high_ids = list(data_test['id'].loc[data_test['is_high'] == 1])
low_ids = list(data_test['id'].loc[data_test['is_high'] == 0])

In [38]:
from catboost import CatBoostRegressor

In [39]:
#первый вариант
model_high_lgb = lgb.LGBMRegressor(
                        n_estimators=350,
                        learning_rate=0.1,
                        feature_fraction=0.7,
                        subsample=0.4,
                        num_leaves=40)    
model_low_lgb = lgb.LGBMRegressor(
                        n_estimators=350,
                        learning_rate=0.1,
                        feature_fraction=0.7,
                        subsample=0.4,
                        num_leaves=40) 
#второй вариант
model_high_catboost = CatBoostRegressor(iterations=350,
                        learning_rate=0.1,
                        depth=10) 
model_low_catboost = CatBoostRegressor(iterations=350,
                        learning_rate=0.1,
                        depth=10)

#третий вариант
model_high_xgb = xgb.XGBRegressor(n_estimators=350,
                        learning_rate=0.1) 
model_low_xgb = xgb.XGBRegressor(n_estimators=350,
                        learning_rate=0.1)


In [40]:
y_train_high = data_high.ln_demand
X_train_high = data_high.drop(['id','ln_demand','is_high', 'demand'], axis=1)


y_train_low = data_low.ln_demand
X_train_low = data_low.drop(['id','ln_demand','is_high', 'demand'], axis=1)

In [41]:
#обучим lgbm 
CV(X_train_high, y_train_high, 20, model_high_lgb, mean_absolute_error, kwargs)
CV(X_train_low, y_train_low, 20, model_low_lgb, mean_absolute_error, kwargs)

/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecat


[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[2]	valid_0's l2: 0.0918618
[4]	valid_0's l2: 0.0913964
[6]	valid_0's l2: 0.0878625
[8]	valid_0's l2: 0.0864607
[10]	valid_0's l2: 0.0865616
[12]	valid_0's l2: 0.0853221
[14]	valid_0's l2: 0.0853148
[16]	valid_0's l2: 0.0846156
[18]	valid_0's l2: 0.0835943
[20]	valid_0's l2: 0.083528
[22]	valid_0's l2: 0.0821346
[24]	valid_0's l2: 0.0819836
[26]	valid_0's l2: 0.0819982
[28]	valid_0's l2: 0.0817985
[30]	valid_0's l2: 0.0817482
[32]	valid_0's l2: 0.0815066
[34]	valid_0's l2: 0.0819783
[36]	valid_0's l2: 0.08197
[38]	valid_0's l2: 0.0817647
[40]	valid_0's l2: 0.0819775

[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[2]	valid_0's l2: 0.0292706
[4]	valid_0's l2: 0.0294594
[6]	valid_0's l2: 0.029607
[8]	valid_0's l2: 0.0296691
[10]	valid_0's l2: 0.0295589
[12]	valid_0's l2: 0.0299234

[LightG

/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecat

[6]	valid_0's l2: 0.0541436
[8]	valid_0's l2: 0.0544375
[10]	valid_0's l2: 0.0546619
[12]	valid_0's l2: 0.0547906
[14]	valid_0's l2: 0.0556324

[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[2]	valid_0's l2: 0.0296785
[4]	valid_0's l2: 0.0288384
[6]	valid_0's l2: 0.0281868
[8]	valid_0's l2: 0.027154
[10]	valid_0's l2: 0.0266893
[12]	valid_0's l2: 0.0266402
[14]	valid_0's l2: 0.0265504
[16]	valid_0's l2: 0.0264307
[18]	valid_0's l2: 0.0262097
[20]	valid_0's l2: 0.0263363
[22]	valid_0's l2: 0.0264829
[24]	valid_0's l2: 0.0267362
[26]	valid_0's l2: 0.0269314
[28]	valid_0's l2: 0.0268858

[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[2]	valid_0's l2: 0.0423152
[4]	valid_0's l2: 0.0414316
[6]	valid_0's l2: 0.0412269
[8]	valid_0's l2: 0.0413101
[10]	valid_0's l2: 0.0421438
[12]	valid_0's l2: 0.0428435
[14]	valid_0's l2: 0.0433171
[16]	va

/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecat

...

/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecat

[12]	valid_0's l2: 0.00837064
[14]	valid_0's l2: 0.00808298
[16]	valid_0's l2: 0.00780546
[18]	valid_0's l2: 0.00772975
[20]	valid_0's l2: 0.00756003
[22]	valid_0's l2: 0.0074054
[24]	valid_0's l2: 0.00731968
[26]	valid_0's l2: 0.00727523
[28]	valid_0's l2: 0.00718182
[30]	valid_0's l2: 0.0071934
[32]	valid_0's l2: 0.00716662
[34]	valid_0's l2: 0.00709537
[36]	valid_0's l2: 0.00709048
[38]	valid_0's l2: 0.00710899
[40]	valid_0's l2: 0.00714568
[42]	valid_0's l2: 0.00715257
[44]	valid_0's l2: 0.00713785
[46]	valid_0's l2: 0.00709458
[48]	valid_0's l2: 0.00708086
[50]	valid_0's l2: 0.00705264
[52]	valid_0's l2: 0.00704679
[54]	valid_0's l2: 0.00703363
[56]	valid_0's l2: 0.00702662
[58]	valid_0's l2: 0.00703142
[60]	valid_0's l2: 0.00703978
[62]	valid_0's l2: 0.00706088

[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[2]	valid_0's l2: 0.0180864
[4]	valid_0's l2: 0.0160836
[6]	valid_0's l2: 0.0146848
[8]	valid_0's

/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecat

[2]	valid_0's l2: 0.0137159
[4]	valid_0's l2: 0.0117808
[6]	valid_0's l2: 0.0104007
[8]	valid_0's l2: 0.0095938
[10]	valid_0's l2: 0.00890457
[12]	valid_0's l2: 0.00838257
[14]	valid_0's l2: 0.00790799
[16]	valid_0's l2: 0.0075972
[18]	valid_0's l2: 0.00736544
[20]	valid_0's l2: 0.00720293
[22]	valid_0's l2: 0.00708471
[24]	valid_0's l2: 0.0069922
[26]	valid_0's l2: 0.00693777
[28]	valid_0's l2: 0.00685988
[30]	valid_0's l2: 0.00683389
[32]	valid_0's l2: 0.00675192
[34]	valid_0's l2: 0.00669196
[36]	valid_0's l2: 0.0065974
[38]	valid_0's l2: 0.00656597
[40]	valid_0's l2: 0.0065697
[42]	valid_0's l2: 0.00657194
[44]	valid_0's l2: 0.00655992
[46]	valid_0's l2: 0.00657917
[48]	valid_0's l2: 0.00655867
[50]	valid_0's l2: 0.00654193
[52]	valid_0's l2: 0.00653596
[54]	valid_0's l2: 0.00652313
[56]	valid_0's l2: 0.00647977
[58]	valid_0's l2: 0.00646839
[60]	valid_0's l2: 0.00646598
[62]	valid_0's l2: 0.0064491
[64]	valid_0's l2: 0.00642978
[66]	valid_0's l2: 0.00641589
[68]	valid_0's l2: 0.00

0.056834986957821944

In [42]:
test_high = data_test.loc[data_test['is_high'] == 1]
test_low = data_test.loc[data_test['is_high'] == 0]

In [43]:
pred_high = model_high_lgb.predict(test_high.drop(['id','demand', 'ln_demand', 'is_high'], axis=1))
pred_low = model_low_lgb.predict(test_low.drop(['id','demand', 'ln_demand', 'is_high'], axis=1))

In [44]:
high = pd.DataFrame({'id': high_ids, 'demand': pred_high})
low = pd.DataFrame({'id': low_ids, 'demand': pred_low})
pred = pd.concat([high, low])

In [45]:
lgb_pred = pd.DataFrame({'id': test_id_list})
lgb_pred = lgb_pred.merge(pred, on = 'id')
lgb_pred['demand'] = np.exp(lgb_pred['demand']) - 1
lgb_pred.head()

,id,demand
0,601,0.173508
1,697,0.180302
2,831,1.045946
3,875,1.132065
4,975,0.174993


In [46]:
lgb_pred.to_csv('lgb_pred.csv', index=False) 

In [47]:
#проверим второй вариант 
data_test['is_high'] = demand_pred_catboost

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [48]:
high_ids = list(data_test['id'].loc[data_test['is_high'] == 1])
low_ids = list(data_test['id'].loc[data_test['is_high'] == 0])

In [49]:
y_train_high = data_high.ln_demand
X_train_high = data_high.drop(['id','ln_demand','is_high', 'demand'], axis=1)


y_train_low = data_low.ln_demand
X_train_low = data_low.drop(['id','ln_demand','is_high', 'demand'], axis=1)

In [50]:
#обучим catboost 
CV(X_train_high, y_train_high, 20, model_high_catboost, mean_absolute_error, kwargs)
CV(X_train_low, y_train_low, 20, model_low_catboost, mean_absolute_error, kwargs)


0:	learn: 0.1500774	test: 0.3099838	best: 0.3099838 (0)	total: 12.4ms	remaining: 4.32s
2:	learn: 0.1435796	test: 0.3083610	best: 0.3083610 (2)	total: 30.3ms	remaining: 3.51s
4:	learn: 0.1374623	test: 0.3026835	best: 0.3026835 (4)	total: 48.7ms	remaining: 3.36s
6:	learn: 0.1318229	test: 0.3007254	best: 0.3007254 (6)	total: 69.3ms	remaining: 3.39s
8:	learn: 0.1264058	test: 0.2966795	best: 0.2966795 (8)	total: 85.2ms	remaining: 3.23s
10:	learn: 0.1228103	test: 0.2952818	best: 0.2952818 (10)	total: 98.2ms	remaining: 3.02s
12:	learn: 0.1189981	test: 0.2956389	best: 0.2951161 (11)	total: 108ms	remaining: 2.8s
14:	learn: 0.1164125	test: 0.2947229	best: 0.2947229 (14)	total: 122ms	remaining: 2.73s
16:	learn: 0.1138709	test: 0.2937278	best: 0.2937278 (16)	total: 137ms	remaining: 2.68s
18:	learn: 0.1119172	test: 0.2931167	best: 0.2925864 (17)	total: 150ms	remaining: 2.61s
20:	learn: 0.1099620	test: 0.2923234	best: 0.2918862 (19)	total: 158ms	remaining: 2.47s
22:	learn: 0.1075525	test: 0.2904850

0.05814869857097763

In [51]:
test_high = data_test.loc[data_test['is_high'] == 1]
test_low = data_test.loc[data_test['is_high'] == 0]

pred_high = model_high_catboost.predict(test_high.drop(['id','demand', 'ln_demand', 'is_high'], axis=1))
pred_low = model_low_catboost.predict(test_low.drop(['id','demand', 'ln_demand', 'is_high'], axis=1))

In [52]:
high = pd.DataFrame({'id': high_ids, 'demand': pred_high})
low = pd.DataFrame({'id': low_ids, 'demand': pred_low})
pred = pd.concat([high, low])

catboost_pred = pd.DataFrame({'id': test_id_list})
catboost_pred = catboost_pred.merge(pred, on = 'id')
catboost_pred['demand'] = np.exp(catboost_pred['demand']) - 1
catboost_pred.head()

,id,demand
0,601,1.068612
1,697,0.154078
2,831,1.033194
3,875,1.026648
4,975,1.051224


In [53]:
catboost_pred.to_csv('catboost_pred.csv', index=False) 
#стало лучше на тесте 

In [54]:
#проверим третий вариант 
data_test['is_high'] = demand_pred_xgb

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [55]:
high_ids = list(data_test['id'].loc[data_test['is_high'] == 1])
low_ids = list(data_test['id'].loc[data_test['is_high'] == 0])

In [56]:
y_train_high = data_high.ln_demand
X_train_high = data_high.drop(['id','ln_demand','is_high', 'demand'], axis=1)


y_train_low = data_low.ln_demand
X_train_low = data_low.drop(['id','ln_demand','is_high', 'demand'], axis=1)

In [57]:
 #обучим xgb
CV(X_train_high, y_train_high, 20, model_high_xgb, mean_absolute_error, kwargs)
CV(X_train_low, y_train_low, 20, model_low_xgb, mean_absolute_error, kwargs)


[0]	validation_0-rmse:0.40634
[2]	validation_0-rmse:0.37615
[4]	validation_0-rmse:0.36165
[6]	validation_0-rmse:0.35869
[8]	validation_0-rmse:0.36279
[10]	validation_0-rmse:0.37145
[12]	validation_0-rmse:0.37085
[14]	validation_0-rmse:0.37283
[16]	validation_0-rmse:0.37539

[0]	validation_0-rmse:0.27966


/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:797: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  UserWarning,
/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:797: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  UserWarning,


[2]	validation_0-rmse:0.24940
[4]	validation_0-rmse:0.22704
[6]	validation_0-rmse:0.21143
[8]	validation_0-rmse:0.19779
[10]	validation_0-rmse:0.18816
[12]	validation_0-rmse:0.18053
[14]	validation_0-rmse:0.17716
[16]	validation_0-rmse:0.17427
[18]	validation_0-rmse:0.17292
[20]	validation_0-rmse:0.17019
[22]	validation_0-rmse:0.16917
[24]	validation_0-rmse:0.16942
[26]	validation_0-rmse:0.16988
[28]	validation_0-rmse:0.17065
[30]	validation_0-rmse:0.17274
[32]	validation_0-rmse:0.17272

[0]	validation_0-rmse:0.25347
[2]	validation_0-rmse:0.22244
[4]	validation_0-rmse:0.19766
[6]	validation_0-rmse:0.17905
[8]	validation_0-rmse:0.16589
[10]	validation_0-rmse:0.15609
[12]	validation_0-rmse:0.14944
[14]	validation_0-rmse:0.14502
[16]	validation_0-rmse:0.14315
[18]	validation_0-rmse:0.14175
[20]	validation_0-rmse:0.14113
[22]	validation_0-rmse:0.14101
[24]	validation_0-rmse:0.14104


/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:797: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  UserWarning,


[26]	validation_0-rmse:0.14130
[28]	validation_0-rmse:0.14157
[30]	validation_0-rmse:0.14263
[31]	validation_0-rmse:0.14279

[0]	validation_0-rmse:0.32340
[2]	validation_0-rmse:0.29138
[4]	validation_0-rmse:0.26896
[6]	validation_0-rmse:0.25282
[8]	validation_0-rmse:0.24179
[10]	validation_0-rmse:0.23219
[12]	validation_0-rmse:0.22735
[14]	validation_0-rmse:0.22383
[16]	validation_0-rmse:0.22150
[18]	validation_0-rmse:0.21983
[20]	validation_0-rmse:0.21863
[22]	validation_0-rmse:0.21804


...

/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:797: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  UserWarning,


[12]	validation_0-rmse:0.13499
[14]	validation_0-rmse:0.12169
[16]	validation_0-rmse:0.11230
[18]	validation_0-rmse:0.10556
[20]	validation_0-rmse:0.10050
[22]	validation_0-rmse:0.09732
[24]	validation_0-rmse:0.09482
[26]	validation_0-rmse:0.09308
[28]	validation_0-rmse:0.09185
[30]	validation_0-rmse:0.09089
[32]	validation_0-rmse:0.09015
[34]	validation_0-rmse:0.08976
[36]	validation_0-rmse:0.08945
[38]	validation_0-rmse:0.08890
[40]	validation_0-rmse:0.08885
[42]	validation_0-rmse:0.08855
[44]	validation_0-rmse:0.08848
[46]	validation_0-rmse:0.08820
[48]	validation_0-rmse:0.08803
[50]	validation_0-rmse:0.08789
[52]	validation_0-rmse:0.08777
[54]	validation_0-rmse:0.08796
[56]	validation_0-rmse:0.08797
[58]	validation_0-rmse:0.08805
[60]	validation_0-rmse:0.08805

[0]	validation_0-rmse:0.38831
[2]	validation_0-rmse:0.31714
[4]	validation_0-rmse:0.26037
[6]	validation_0-rmse:0.21547
[8]	validation_0-rmse:0.18113
[10]	validation_0-rmse:0.15416


/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:797: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  UserWarning,


[12]	validation_0-rmse:0.13337
[14]	validation_0-rmse:0.11804
[16]	validation_0-rmse:0.10723
[18]	validation_0-rmse:0.09922
[20]	validation_0-rmse:0.09397
[22]	validation_0-rmse:0.08991
[24]	validation_0-rmse:0.08733
[26]	validation_0-rmse:0.08558
[28]	validation_0-rmse:0.08424
[30]	validation_0-rmse:0.08353
[32]	validation_0-rmse:0.08273
[34]	validation_0-rmse:0.08219
[36]	validation_0-rmse:0.08183
[38]	validation_0-rmse:0.08142
[40]	validation_0-rmse:0.08120
[42]	validation_0-rmse:0.08080
[44]	validation_0-rmse:0.08070
[46]	validation_0-rmse:0.08064
[48]	validation_0-rmse:0.08061
[50]	validation_0-rmse:0.08049
[52]	validation_0-rmse:0.08039
[54]	validation_0-rmse:0.08027
[56]	validation_0-rmse:0.08021
[58]	validation_0-rmse:0.08019
[60]	validation_0-rmse:0.08010
[62]	validation_0-rmse:0.08010
[64]	validation_0-rmse:0.07994
[66]	validation_0-rmse:0.07986
[68]	validation_0-rmse:0.07969
[70]	validation_0-rmse:0.07963
[72]	validation_0-rmse:0.07959
[74]	validation_0-rmse:0.07959
[76]	val

0.06196236787126579

In [58]:
test_high = data_test.loc[data_test['is_high'] == 1]
test_low = data_test.loc[data_test['is_high'] == 0]

pred_high = model_high_xgb.predict(test_high.drop(['id','demand', 'ln_demand', 'is_high'], axis=1))
pred_low = model_low_xgb.predict(test_low.drop(['id','demand', 'ln_demand', 'is_high'], axis=1))

In [59]:
high = pd.DataFrame({'id': high_ids, 'demand': pred_high})
low = pd.DataFrame({'id': low_ids, 'demand': pred_low})
pred = pd.concat([high, low])

xgb_pred = pd.DataFrame({'id': test_id_list})
xgb_pred = xgb_pred.merge(pred, on = 'id')
xgb_pred['demand'] = np.exp(xgb_pred['demand']) - 1
xgb_pred.head()

,id,demand
0,601,1.048347
1,697,1.061058
2,831,1.076410
3,875,1.238069
4,975,1.032526


In [60]:
xgb_pred.to_csv('xgb_pred.csv', index=False) 
#работает хуже всего

In [61]:
#возьмем катбуст как классификатор и lgbm как регрессор 
data_test['is_high'] = demand_pred_catboost

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [62]:
high_ids = list(data_test['id'].loc[data_test['is_high'] == 1])
low_ids = list(data_test['id'].loc[data_test['is_high'] == 0])

y_train_high = data_high.ln_demand
X_train_high = data_high.drop(['id','ln_demand','is_high', 'demand'], axis=1)

y_train_low = data_low.ln_demand
X_train_low = data_low.drop(['id','ln_demand','is_high', 'demand'], axis=1)

In [63]:
model_high_lgb = lgb.LGBMRegressor(
                        n_estimators=350,
                        learning_rate=0.1,
                        feature_fraction=0.7,
                        subsample=0.4,
                        num_leaves=40)    
model_low_lgb = lgb.LGBMRegressor(
                        n_estimators=350,
                        learning_rate=0.1,
                        feature_fraction=0.7,
                        subsample=0.4,
                        num_leaves=40) 

model_high_catboost = CatBoostRegressor(iterations=350,
                        learning_rate=0.1,
                        depth=10) 
model_low_catboost = CatBoostRegressor(iterations=350,
                        learning_rate=0.1,
                        depth=10)

In [64]:
#обучим lgbm
CV(X_train_high, y_train_high, 20, model_high_lgb, mean_absolute_error, kwargs)
CV(X_train_low, y_train_low, 20, model_low_lgb, mean_absolute_error, kwargs)

/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecat


[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[2]	valid_0's l2: 0.0918618
[4]	valid_0's l2: 0.0913964
[6]	valid_0's l2: 0.0878625
[8]	valid_0's l2: 0.0864607
[10]	valid_0's l2: 0.0865616
[12]	valid_0's l2: 0.0853221
[14]	valid_0's l2: 0.0853148
[16]	valid_0's l2: 0.0846156
[18]	valid_0's l2: 0.0835943
[20]	valid_0's l2: 0.083528
[22]	valid_0's l2: 0.0821346
[24]	valid_0's l2: 0.0819836
[26]	valid_0's l2: 0.0819982
[28]	valid_0's l2: 0.0817985
[30]	valid_0's l2: 0.0817482
[32]	valid_0's l2: 0.0815066
[34]	valid_0's l2: 0.0819783
[36]	valid_0's l2: 0.08197
[38]	valid_0's l2: 0.0817647
[40]	valid_0's l2: 0.0819775

[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[2]	valid_0's l2: 0.0292706
[4]	valid_0's l2: 0.0294594
[6]	valid_0's l2: 0.029607
[8]	valid_0's l2: 0.0296691
[10]	valid_0's l2: 0.0295589
[12]	valid_0's l2: 0.0299234

[LightG

/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecat

[4]	valid_0's l2: 0.0543689
[6]	valid_0's l2: 0.0541436
[8]	valid_0's l2: 0.0544375
[10]	valid_0's l2: 0.0546619
[12]	valid_0's l2: 0.0547906
[14]	valid_0's l2: 0.0556324

[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[2]	valid_0's l2: 0.0296785
[4]	valid_0's l2: 0.0288384
[6]	valid_0's l2: 0.0281868
[8]	valid_0's l2: 0.027154
[10]	valid_0's l2: 0.0266893
[12]	valid_0's l2: 0.0266402
[14]	valid_0's l2: 0.0265504
[16]	valid_0's l2: 0.0264307
[18]	valid_0's l2: 0.0262097
[20]	valid_0's l2: 0.0263363
[22]	valid_0's l2: 0.0264829
[24]	valid_0's l2: 0.0267362
[26]	valid_0's l2: 0.0269314
[28]	valid_0's l2: 0.0268858

[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[2]	valid_0's l2: 0.0423152
[4]	valid_0's l2: 0.0414316
[6]	valid_0's l2: 0.0412269
[8]	valid_0's l2: 0.0413101
[10]	valid_0's l2: 0.0421438
[12]	valid_0's l2: 0.0428435
[14]	val

/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecat

...

[4]	valid_0's l2: 0.01506
[6]	valid_0's l2: 0.0140044
[8]	valid_0's l2: 0.0135362
[10]	valid_0's l2: 0.0131475
[12]	valid_0's l2: 0.0128503
[14]	valid_0's l2: 0.0126538
[16]	valid_0's l2: 0.0124636
[18]	valid_0's l2: 0.0124636
[20]	valid_0's l2: 0.0123984
[22]	valid_0's l2: 0.0122742
[24]	valid_0's l2: 0.0121498
[26]	valid_0's l2: 0.0121309
[28]	valid_0's l2: 0.0121141
[30]	valid_0's l2: 0.0121804
[32]	valid_0's l2: 0.0121594
[34]	valid_0's l2: 0.0121609
[36]	valid_0's l2: 0.0121525
[38]	valid_0's l2: 0.01216

[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[2]	valid_0's l2: 0.0129593
[4]	valid_0's l2: 0.011116
[6]	valid_0's l2: 0.00977788
[8]	valid_0's l2: 0.00929349
[10]	valid_0's l2: 0.00864035
[12]	valid_0's l2: 0.00837064
[14]	valid_0's l2: 0.00808298
[16]	valid_0's l2: 0.00780546
[18]	valid_0's l2: 0.00772975
[20]	valid_0's l2: 0.00756003
[22]	valid_0's l2: 0.0074054
[24]	valid_0's l2: 0.00731968
[26]	val

/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecat

[4]	valid_0's l2: 0.0118808
[6]	valid_0's l2: 0.0105007
[8]	valid_0's l2: 0.00998196
[10]	valid_0's l2: 0.00943502
[12]	valid_0's l2: 0.00898239
[14]	valid_0's l2: 0.00851277
[16]	valid_0's l2: 0.00820502
[18]	valid_0's l2: 0.00815941
[20]	valid_0's l2: 0.0080796
[22]	valid_0's l2: 0.00796615
[24]	valid_0's l2: 0.00786245
[26]	valid_0's l2: 0.00783219
[28]	valid_0's l2: 0.0077852
[30]	valid_0's l2: 0.00776946
[32]	valid_0's l2: 0.00774472
[34]	valid_0's l2: 0.00775472
[36]	valid_0's l2: 0.00776351
[38]	valid_0's l2: 0.0077612
[40]	valid_0's l2: 0.00776575
[42]	valid_0's l2: 0.00778935

[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[2]	valid_0's l2: 0.0137159
[4]	valid_0's l2: 0.0117808
[6]	valid_0's l2: 0.0104007
[8]	valid_0's l2: 0.0095938
[10]	valid_0's l2: 0.00890457
[12]	valid_0's l2: 0.00838257
[14]	valid_0's l2: 0.00790799
[16]	valid_0's l2: 0.0075972
[18]	valid_0's l2: 0.00736544
[20]	valid_0's l2: 0.0

/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


0.056834986957821944

In [65]:
test_high = data_test.loc[data_test['is_high'] == 1]
test_low = data_test.loc[data_test['is_high'] == 0]

pred_high = model_high_lgb.predict(test_high.drop(['id','demand', 'ln_demand', 'is_high'], axis=1))
pred_low = model_low_lgb.predict(test_low.drop(['id','demand', 'ln_demand', 'is_high'], axis=1))

In [66]:
high = pd.DataFrame({'id': high_ids, 'demand': pred_high})
low = pd.DataFrame({'id': low_ids, 'demand': pred_low})
pred = pd.concat([high, low])

pred1 = pd.DataFrame({'id': test_id_list})
pred1 = pred1.merge(pred, on = 'id')
pred1['demand'] = np.exp(pred1['demand']) - 1
pred1.head()

,id,demand
0,601,1.048376
1,697,0.180302
2,831,1.045946
3,875,1.132065
4,975,1.042816


In [67]:
pred1.to_csv('lgbm_cat_pred1.csv', index=False) 
#работает лучше, чем все предыдущие варианты, при добавлении признаков promo и actual price
#работает хуже, чем обычный катбуст, если их не добавлять

In [68]:
#возьмем катбуст как регрессор и lgbm как классификатор
data_test['is_high'] = demand_pred_lgbm

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [69]:
high_ids = list(data_test['id'].loc[data_test['is_high'] == 1])
low_ids = list(data_test['id'].loc[data_test['is_high'] == 0])

y_train_high = data_high.ln_demand
X_train_high = data_high.drop(['id','ln_demand','is_high', 'demand'], axis=1)

y_train_low = data_low.ln_demand
X_train_low = data_low.drop(['id','ln_demand','is_high', 'demand'], axis=1)

In [70]:
#обучим catboost
CV(X_train_high, y_train_high, 20, model_high_catboost, mean_absolute_error, kwargs)
CV(X_train_low, y_train_low, 20, model_low_catboost, mean_absolute_error, kwargs)


0:	learn: 0.1500774	test: 0.3099838	best: 0.3099838 (0)	total: 6.45ms	remaining: 2.25s
2:	learn: 0.1435796	test: 0.3083610	best: 0.3083610 (2)	total: 19.8ms	remaining: 2.29s
4:	learn: 0.1374623	test: 0.3026835	best: 0.3026835 (4)	total: 31.7ms	remaining: 2.19s
6:	learn: 0.1318229	test: 0.3007254	best: 0.3007254 (6)	total: 43.1ms	remaining: 2.11s
8:	learn: 0.1264058	test: 0.2966795	best: 0.2966795 (8)	total: 58.5ms	remaining: 2.22s
10:	learn: 0.1228103	test: 0.2952818	best: 0.2952818 (10)	total: 77.9ms	remaining: 2.4s
12:	learn: 0.1189981	test: 0.2956389	best: 0.2951161 (11)	total: 85.6ms	remaining: 2.22s
14:	learn: 0.1164125	test: 0.2947229	best: 0.2947229 (14)	total: 98ms	remaining: 2.19s
16:	learn: 0.1138709	test: 0.2937278	best: 0.2937278 (16)	total: 110ms	remaining: 2.16s
18:	learn: 0.1119172	test: 0.2931167	best: 0.2925864 (17)	total: 124ms	remaining: 2.15s
20:	learn: 0.1099620	test: 0.2923234	best: 0.2918862 (19)	total: 132ms	remaining: 2.06s
22:	learn: 0.1075525	test: 0.2904850

0.05814869857097763

In [71]:
test_high = data_test.loc[data_test['is_high'] == 1]
test_low = data_test.loc[data_test['is_high'] == 0]

pred_high = model_high_catboost.predict(test_high.drop(['id','demand', 'ln_demand', 'is_high'], axis=1))
pred_low = model_low_catboost.predict(test_low.drop(['id','demand', 'ln_demand', 'is_high'], axis=1))

In [72]:
high = pd.DataFrame({'id': high_ids, 'demand': pred_high})
low = pd.DataFrame({'id': low_ids, 'demand': pred_low})
pred = pd.concat([high, low])

pred2 = pd.DataFrame({'id': test_id_list})
pred2 = pred2.merge(pred, on = 'id')
pred2['demand'] = np.exp(pred2['demand']) - 1
pred2.head()

,id,demand
0,601,0.166299
1,697,0.154078
2,831,1.033194
3,875,1.026648
4,975,0.138747


In [73]:
pred2.to_csv('lgbm_cat_pred2.csv', index=False) 
#работает хуже

#лучший результат при добавлении признаков promo, actual price: lgbm регрессор + catboost классификатор
#лучший результат: catboost регрессор + catboost классификатор